In [1]:
import os
os.chdir('../')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data_train_path = "Dataset/data_train.csv"

URM_all_dataframe = pd.read_csv(data_train_path)
print(URM_all_dataframe.head(10))

   user_id  item_id  data
0        0        0   1.0
1        0        2   1.0
2        0      120   1.0
3        0      128   1.0
4        0      211   1.0
5        0      232   1.0
6        0      282   1.0
7        0      453   1.0
8        0      458   1.0
9        0      491   1.0


# Splitting 70/20/10 

In [3]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 456 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


# Defining recommender

In [4]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender


def objective_function(trial):

    params = {
        'topK': trial.suggest_int('topK', 5, 1000),
        'alpha': trial.suggest_float('alpha', 0, 2),
        'beta': trial.suggest_float('beta', 0, 2),
    }


    recommender = RP3betaRecommender(URM_train)

    recommender.fit(**params)

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [5]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/RP3/rp3_firstTrail.csv", index = False)

In [6]:
import optuna

optuna_study = optuna.create_study(study_name="rp3", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2024-12-23 18:18:07,534] A new study created in memory with name: rp3


RP3betaRecommender: Similarity column 38121 (100.0%), 3198.73 column/sec. Elapsed time 11.92 sec
EvaluatorHoldout: Processed 35280 (100.0%) in 13.51 sec. Users per second: 2611


[I 2024-12-23 18:18:36,131] Trial 0 finished with value: 0.016675441142065425 and parameters: {'topK': 139, 'alpha': 0.6145139516361975, 'beta': 0.9092745480766007}. Best is trial 0 with value: 0.016675441142065425.


RP3betaRecommender: Similarity column 38121 (100.0%), 1365.52 column/sec. Elapsed time 27.92 sec
EvaluatorHoldout: Processed 35280 (100.0%) in 20.56 sec. Users per second: 1716


[I 2024-12-23 18:19:40,932] Trial 1 finished with value: 0.02646666914120053 and parameters: {'topK': 775, 'alpha': 0.3193621483681308, 'beta': 0.2681333676809894}. Best is trial 1 with value: 0.02646666914120053.


...

[I 2024-12-23 19:14:44,052] Trial 97 finished with value: 0.030153451526112172 and parameters: {'topK': 55, 'alpha': 0.3434352639325443, 'beta': 0.08206428572607197}. Best is trial 94 with value: 0.03131802271172927.


RP3betaRecommender: Similarity column 38121 (100.0%), 3705.95 column/sec. Elapsed time 10.29 sec
EvaluatorHoldout: Processed 35280 (100.0%) in 10.84 sec. Users per second: 3254


[I 2024-12-23 19:15:05,941] Trial 98 finished with value: 0.031195181180936973 and parameters: {'topK': 23, 'alpha': 0.42503711832305624, 'beta': 0.2930682805258472}. Best is trial 94 with value: 0.03131802271172927.


RP3betaRecommender: Similarity column 38121 (100.0%), 3558.99 column/sec. Elapsed time 10.71 sec
EvaluatorHoldout: Processed 35280 (100.0%) in 10.62 sec. Users per second: 3322


[I 2024-12-23 19:15:27,968] Trial 99 finished with value: 0.0314285849260329 and parameters: {'topK': 22, 'alpha': 0.4294843699758765, 'beta': 0.18002013067989414}. Best is trial 99 with value: 0.0314285849260329.


In [7]:
# Let's print the best hyperparameters we found
print("Best hyperparameters found: " + str(optuna_study.best_params))

Best hyperparameters found: {'topK': 22, 'alpha': 0.4294843699758765, 'beta': 0.18002013067989414}


In [9]:
# Performance on test
recommender = RP3betaRecommender(URM_train)
recommender.fit(**optuna_study.best_params)

result_dict, _ = evaluator_test.evaluateRecommender(recommender)

MAP = result_dict.loc[10]["MAP"]

print("MAP on test: "+str(MAP))

RP3betaRecommender: Similarity column 38121 (100.0%), 3779.08 column/sec. Elapsed time 10.09 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 11.18 sec. Users per second: 3185
MAP on test: 0.03990267381944437


In [12]:
# Let's see also considering validation
recommender = RP3betaRecommender(URM_train_validation)
recommender.fit(**optuna_study.best_params)
result_dict, _ = evaluator_test.evaluateRecommender(recommender)

MAP = result_dict.loc[10]["MAP"]

print("MAP on test considering validation: "+str(MAP))

RP3betaRecommender: Similarity column 38121 (100.0%), 3550.07 column/sec. Elapsed time 10.74 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 11.14 sec. Users per second: 3195
MAP on test considering validation: 0.05377804199358983


# Further parameters searching
Based on the above results, let's facilitate the search of the best hyperparameters in a smaller hyperparameters space

In [13]:
def objective_function_smaller(trial):
    params = {
        'topK': trial.suggest_int('topK', 5, 300),
        'alpha': trial.suggest_float('alpha', 0.3, 1.5),
        'beta': trial.suggest_float('beta', 0.1, 1.0),
    }

    recommender = RP3betaRecommender(URM_train)
    recommender.fit(**params)
    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP = result_dict.loc[10]["MAP"]
    
    return MAP

In [14]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/RP3/rp3_smallerHyperSpace.csv", index = False)

In [15]:
import optuna

optuna_study = optuna.create_study(study_name="rp3_smaller", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2024-12-23 19:33:49,366] A new study created in memory with name: rp3_smaller


RP3betaRecommender: Similarity column 38121 (100.0%), 1360.99 column/sec. Elapsed time 28.01 sec
EvaluatorHoldout: Processed 35280 (100.0%) in 24.57 sec. Users per second: 1436


[I 2024-12-23 19:34:56,428] Trial 0 finished with value: 0.022322091071878047 and parameters: {'topK': 421, 'alpha': 0.8693959588116751, 'beta': 0.7019254269166117}. Best is trial 0 with value: 0.022322091071878047.


RP3betaRecommender: Similarity column 38121 (100.0%), 1635.95 column/sec. Elapsed time 23.30 sec
EvaluatorHoldout: Processed 35280 (100.0%) in 22.08 sec. Users per second: 1598


[I 2024-12-23 19:35:51,649] Trial 1 finished with value: 0.009799997975380819 and parameters: {'topK': 322, 'alpha': 1.772213689105113, 'beta': 1.0085990087846484}. Best is trial 0 with value: 0.022322091071878047.


...

[I 2024-12-23 20:27:51,178] Trial 97 finished with value: 0.03006670220998365 and parameters: {'topK': 62, 'alpha': 0.237358667684739, 'beta': 0.21829969332394913}. Best is trial 95 with value: 0.031600887908072334.


RP3betaRecommender: Similarity column 38121 (100.0%), 4097.63 column/sec. Elapsed time 9.30 sec
EvaluatorHoldout: Processed 35280 (100.0%) in 11.34 sec. Users per second: 3110


[I 2024-12-23 20:28:12,947] Trial 98 finished with value: 0.022639081227728786 and parameters: {'topK': 46, 'alpha': 0.04002780576826248, 'beta': 0.7076822601272551}. Best is trial 95 with value: 0.031600887908072334.


RP3betaRecommender: Similarity column 38121 (100.0%), 1199.96 column/sec. Elapsed time 31.77 sec
EvaluatorHoldout: Processed 35280 (100.0%) in 21.15 sec. Users per second: 1668


[I 2024-12-23 20:29:23,609] Trial 99 finished with value: 0.02259943130691376 and parameters: {'topK': 925, 'alpha': 0.18667941648392344, 'beta': 0.04184326236692414}. Best is trial 95 with value: 0.031600887908072334.


In [16]:
# Let's print the best hyperparameters we found
print("Best hyperparameters found: " + str(optuna_study.best_params))

Best hyperparameters found: {'topK': 18, 'alpha': 0.35836185552301125, 'beta': 0.14541539336200693}


Let's see if we have better performance than before

In [17]:
# Performance on test
recommender = RP3betaRecommender(URM_train)
recommender.fit(**optuna_study.best_params)

result_dict, _ = evaluator_test.evaluateRecommender(recommender)

MAP = result_dict.loc[10]["MAP"]

print("MAP on test: "+str(MAP))

RP3betaRecommender: Similarity column 38121 (100.0%), 4159.90 column/sec. Elapsed time 9.16 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 10.73 sec. Users per second: 3317
MAP on test: 0.0396921462127938


So we had similar results if not worse, let's find our final values but with URM_train_validation

# Find final parameters
Let's also use k-cross-validatio since easy to compute for this model

In [18]:
k_fold = 10
seeds = [192760, 1393659, 1269293, 138973, 931308, 1088652, 1361151, 1456105, 820535, 1240839]
assert len(seeds) == k_fold

In [19]:
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout



URM_all = sps.coo_matrix((URM_all_dataframe['data'], (URM_all_dataframe['user_id'], URM_all_dataframe['item_id'])))
URM_all = URM_all.tocsr()
urm_trains = []
urm_validations = []
urm_test = []
urm_train_validations = []
# Defining evaluators
evaluator_validation = []
evaluator_test = []

for i in range(k_fold):
    # Splitting data into 64% train, 16% validation, 20% test
    URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=seeds[i])
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8, seed=seeds[i])
    
    urm_trains.append(URM_train)
    urm_validations.append(URM_validation)
    urm_test.append(URM_test)
    urm_train_validations.append(URM_train_validation)
    
    evaluator_validation.append(EvaluatorHoldout(URM_validation, cutoff_list=[10]))
    evaluator_test.append(EvaluatorHoldout(URM_test, cutoff_list=[10]))

EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 447 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 155 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 441 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 167 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 467 ( 1.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 163 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 425 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 169 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 413 ( 1.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 154 ( 0.4%) Users that have less than 

In [23]:
def objective_function_smaller(trial):
    params = {
        'topK': trial.suggest_int('topK', 5, 300),
        'alpha': trial.suggest_float('alpha', 0.3, 1.5),
        'beta': trial.suggest_float('beta', 0.1, 1.0),
    }
    
    MAP = 0
    for i in range(k_fold):
        recommender = RP3betaRecommender(urm_train_validations[i])
        recommender.fit(**params)

        result_dict, _ = evaluator_test[i].evaluateRecommender(recommender)

        MAP += result_dict.loc[10]["MAP"]
    
    MAP /= k_fold
    
    return MAP

In [24]:
class SaveResults(object):
    
    def __init__(self):
        self.results_df = pd.DataFrame(columns = ["result"])
    
    def __call__(self, optuna_study, optuna_trial):
        hyperparam_dict = optuna_trial.params.copy()
        hyperparam_dict["result"] = optuna_trial.values[0]
        
        self.results_df = pd.concat([self.results_df, pd.DataFrame([hyperparam_dict])], ignore_index=True)
        self.results_df.to_csv("logs/RP3/rp3_kcross_trainVal.csv", index = False)

In [25]:
import optuna

optuna_study = optuna.create_study(study_name="rp3_kcross_trainVal", direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_smaller,
                      callbacks=[save_results],
                      n_trials = 100
                      )

[I 2024-12-24 09:05:50,784] A new study created in memory with name: rp3_kcross_trainVal


RP3betaRecommender: Similarity column 38121 (100.0%), 2547.51 column/sec. Elapsed time 14.96 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 15.44 sec. Users per second: 2303
RP3betaRecommender: Similarity column 38121 (100.0%), 2490.57 column/sec. Elapsed time 15.31 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 15.48 sec. Users per second: 2299
RP3betaRecommender: Similarity column 38121 (100.0%), 2634.73 column/sec. Elapsed time 14.47 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 15.40 sec. Users per second: 2310
RP3betaRecommender: Similarity column 38121 (100.0%), 2640.41 column/sec. Elapsed time 14.44 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 15.37 sec. Users per second: 2314
RP3betaRecommender: Similarity column 38121 (100.0%), 2642.87 column/sec. Elapsed time 14.42 sec
EvaluatorHoldout: Processed 35567 (100.0%) in 15.37 sec. Users per second: 2315
RP3betaRecommender: Similarity column 38121 (100.0%), 2493.55 column/sec. Elapsed time 15.29 sec
EvaluatorHoldout: 

[I 2024-12-24 09:11:40,076] Trial 0 finished with value: 0.04835806230888804 and parameters: {'topK': 194, 'alpha': 0.5552078548529443, 'beta': 0.37767419387555407}. Best is trial 0 with value: 0.04835806230888804.


RP3betaRecommender: Similarity column 38121 (100.0%), 2593.22 column/sec. Elapsed time 14.70 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 16.22 sec. Users per second: 2192
RP3betaRecommender: Similarity column 38121 (100.0%), 2572.59 column/sec. Elapsed time 14.82 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 16.14 sec. Users per second: 2204
RP3betaRecommender: Similarity column 38121 (100.0%), 2583.30 column/sec. Elapsed time 14.76 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 16.18 sec. Users per second: 2199
RP3betaRecommender: Similarity column 38121 (100.0%), 2604.70 column/sec. Elapsed time 14.64 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 16.09 sec. Users per second: 2211
RP3betaRecommender: Similarity column 38121 (100.0%), 2601.88 column/sec. Elapsed time 14.65 sec
EvaluatorHoldout: Processed 35567 (100.0%) in 16.12 sec. Users per second: 2207
RP3betaRecommender: Similarity column 38121 (100.0%), 2582.72 column/sec. Elapsed time 14.76 sec
EvaluatorHoldout: 

[I 2024-12-24 09:17:39,966] Trial 1 finished with value: 0.036230258567758813 and parameters: {'topK': 202, 'alpha': 1.479909029333969, 'beta': 0.653164055020753}. Best is trial 0 with value: 0.04835806230888804.


...

[I 2024-12-24 16:21:38,524] Trial 97 finished with value: 0.05117919402346258 and parameters: {'topK': 5, 'alpha': 0.36176200082234294, 'beta': 0.19535549579998518}. Best is trial 42 with value: 0.05517828622068708.


RP3betaRecommender: Similarity column 38121 (100.0%), 2641.15 column/sec. Elapsed time 14.43 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 13.00 sec. Users per second: 2736
RP3betaRecommender: Similarity column 38121 (100.0%), 3047.69 column/sec. Elapsed time 12.51 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 13.02 sec. Users per second: 2733
RP3betaRecommender: Similarity column 38121 (100.0%), 3028.78 column/sec. Elapsed time 12.59 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 13.00 sec. Users per second: 2735
RP3betaRecommender: Similarity column 38121 (100.0%), 2961.03 column/sec. Elapsed time 12.87 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 12.94 sec. Users per second: 2750
RP3betaRecommender: Similarity column 38121 (100.0%), 2934.56 column/sec. Elapsed time 12.99 sec
EvaluatorHoldout: Processed 35567 (100.0%) in 12.95 sec. Users per second: 2746
RP3betaRecommender: Similarity column 38121 (100.0%), 3027.85 column/sec. Elapsed time 12.59 sec
EvaluatorHoldout: 

[I 2024-12-24 16:26:25,639] Trial 98 finished with value: 0.04893226771176491 and parameters: {'topK': 128, 'alpha': 0.41100665638912254, 'beta': 0.22052571642933036}. Best is trial 42 with value: 0.05517828622068708.


RP3betaRecommender: Similarity column 38121 (100.0%), 3343.14 column/sec. Elapsed time 11.40 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 11.30 sec. Users per second: 3146
RP3betaRecommender: Similarity column 38121 (100.0%), 3755.81 column/sec. Elapsed time 10.15 sec
EvaluatorHoldout: Processed 35581 (100.0%) in 11.26 sec. Users per second: 3160
RP3betaRecommender: Similarity column 38121 (100.0%), 3774.41 column/sec. Elapsed time 10.10 sec
EvaluatorHoldout: Processed 35569 (100.0%) in 11.23 sec. Users per second: 3169
RP3betaRecommender: Similarity column 38121 (100.0%), 3817.22 column/sec. Elapsed time 9.99 sec
EvaluatorHoldout: Processed 35573 (100.0%) in 11.28 sec. Users per second: 3153
RP3betaRecommender: Similarity column 38121 (100.0%), 3740.82 column/sec. Elapsed time 10.19 sec
EvaluatorHoldout: Processed 35567 (100.0%) in 11.21 sec. Users per second: 3172
RP3betaRecommender: Similarity column 38121 (100.0%), 3661.47 column/sec. Elapsed time 10.41 sec
EvaluatorHoldout: P

[I 2024-12-24 16:30:11,521] Trial 99 finished with value: 0.05259509025886934 and parameters: {'topK': 36, 'alpha': 0.5145226258808551, 'beta': 0.39859841102867366}. Best is trial 42 with value: 0.05517828622068708.


In [26]:
best_index = save_results.results_df["result"].idxmax()
best_hyperparams = save_results.results_df.loc[best_index].to_dict()
print(best_hyperparams)

{'result': 0.05517828622068708, 'topK': 14.0, 'alpha': 0.3683550822991944, 'beta': 0.19877125816137325}


# Producing the csv with the all dataset

In [27]:
recommender = RP3betaRecommender(URM_all)
best_hyperparams = optuna_study.best_params
recommender.fit(**best_hyperparams)

RP3betaRecommender: Similarity column 38121 (100.0%), 3086.58 column/sec. Elapsed time 12.35 sec


In [29]:
target_user = pd.read_csv("Dataset/data_target_users_test.csv")
print(target_user.head(10))

user_id = target_user['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))

# Saving in a submission file for the competition
submission = pd.DataFrame(columns=['user_id', 'item_list'])
submission['user_id'] = user_id
# Convert list of recommendations into a space-separated string
submission['item_list'] = [' '.join(map(str, recs)) for recs in recommendations]

# Save the submission file
submission.to_csv(f"submissions/rp3_localTestCross55_bestParam_k:{best_hyperparams['topK']}_alpha:{best_hyperparams['alpha']}_beta_{best_hyperparams['beta']}.csv", index=False)

   user_id
0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
